In [ ]:
# Installing dependencies
!pip install yfinance openai autogen plotly

# Importing necessary libraries
import yfinance as yf
import dash
import plotly.graph_objs as go
import openai
from autogen import AssistantAgent,UserProxyAgent

# Setting your OpenAI API key
openai.api_key=""  # Replace it with your OpenAI API key

# Defining stock symbols
STOCKS={
    "Apple(AAPL)":"AAPL",
    "Reliance Industries(RELIANCE.NS)":"RELIANCE.NS",
    "Tata Consultancy Services(TCS.NS)":"TCS.NS",
    "Infosys(INFY.NS)":"INFY.NS",
    "HDFC Bank(HDFCBANK.NS)":"HDFCBANK.NS"
}

# Custom Tool: Fetches stock data for a given period.
def fetch_stock_data(ticker_symbol:str,period:str="1mo")->dict:
    """
    Fetches historical stock data for the given ticker symbol and time period.
    Args:
    ticker_symbol (str):Stock ticker(e.g.,"AAPL"or"RELIANCE.NS").
    period(str):Time period for historical data.
    dict:Stock historical data summary and full dataset.
    """
    stock=yf.Ticker(ticker_symbol)
    hist=stock.history(period=period)
    if hist.empty:
        return{"error":"No data available"}
    return {
        "Stock":ticker_symbol,
        "Historical Data":hist.reset_index().to_dict(orient="list"),
    }

# Defining an Assistant Agent
assistant=AssistantAgent(
    name="StockAgent",
    system_message="I can fetch stock data for you. First,tell me which stock you're interested in.Then,provide a time period."
)

# Registering the function
assistant.register_function({"fetch_stock_data":fetch_stock_data})

# Define a User Proxy Agent
user=UserProxyAgent(name="User",human_input_mode="ALWAYS")

# **Agent Interaction:Asks for Stock,Then Time Period,Then Fetch & Show Graph**
def agent_demo():
    while True:
        print("\n💬 StockAgent: Hello! Which stock are you interested in?")
        stock_input=input("User:") # User enters the stock name (e.g.,AAPL)

        # Validating stock
        if stock_input not in STOCKS.values():
            print("\n❌ StockAgent:Invalid stock symbol.Please choose from:",list(STOCKS.values()))
            continue
        print("\n💬 StockAgent:Great! What time period should I fetch the data for?(e.g.,'1mo','3mo','1y')")
        time_period = input("User: ") # User enters a time period
        print("\n📡 Fetching data,please wait...\n")
        response=user.initiate_chat(
            assistant,
            message=f"Get me the stock price of {stock_input} for {time_period}."
        )

        # Extracting data from agent response
        data=fetch_stock_data(stock_input, time_period)

        if "error" in data:
            print("\n❌ StockAgent:No data available for this request.")
            continue  # Restarting the process

        # Conversation **ends** before visualization happens
        print("\n✅ StockAgent:",response)
        print("\n📊 Here is the stock price graph for your selection:")

        # Visualization using Plotly
        hist=data["Historical Data"]
        fig=go.Figure()
        fig.add_trace(go.Scatter(
            x=hist["Date"], y=hist["Close"],
            mode="lines",name=f"{stock_input}({time_period})"
        ))
        fig.update_layout(
            title=f"Stock Price of {stock_input} over {time_period}",
            xaxis_title="Date",
            yaxis_title="Stock Price (INR/USD)",
            template="plotly_dark"
        )
        fig.show()  # Showing graph **after** the conversation ends.

        # Asking if the user wants another stock data
        print("\n💬 StockAgent: Would you like to check another stock?(yes/no)")
        choice=input("User:").strip().lower()
        if choice!="yes":
            print("\n👋 StockAgent:Okay! Have a great day!")
            break  # Exiting the loop if user says "no"

# Running the agent demo
agent_demo()



💬 StockAgent: Hello! Which stock are you interested in?
User:AAPL

💬 StockAgent:Great! What time period should I fetch the data for?(e.g.,'1mo','3mo','1y')
User: 1mo

📡 Fetching data,please wait...

User (to StockAgent):

Get me the stock price of AAPL for 1mo.

--------------------------------------------------------------------------------
StockAgent (to User):



--------------------------------------------------------------------------------
Replying as User. Provide feedback to StockAgent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (4e832be6-d3bb-4799-970f-f7f204238b3d): User requested to end the conversation

✅ StockAgent: ChatResult(chat_id=None, chat_history=[{'content': 'Get me the stock price of AAPL for 1mo.', 'role': 'assistant', 'name': 'User'}, {'content': '', 'role': 'user', 'name': 'StockAgent'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference


💬 StockAgent: Would you like to check another stock?(yes/no)
User:yes

💬 StockAgent: Hello! Which stock are you interested in?
User:RELIANCE.NS

💬 StockAgent:Great! What time period should I fetch the data for?(e.g.,'1mo','3mo','1y')
User: 3mo

📡 Fetching data,please wait...

User (to StockAgent):

Get me the stock price of RELIANCE.NS for 3mo.

--------------------------------------------------------------------------------
StockAgent (to User):



--------------------------------------------------------------------------------
Replying as User. Provide feedback to StockAgent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (d7db6b56-0ca5-4815-b420-0397a67be291): User requested to end the conversation

✅ StockAgent: ChatResult(chat_id=None, chat_history=[{'content': 'Get me the stock price of RELIANCE.NS for 3mo.', 'role': 'assistant', 'name': 'User'}, {'content': '', 'role': 'user', 'name': 'StockAgent'}], summary='', co


💬 StockAgent: Would you like to check another stock?(yes/no)
User:no

👋 StockAgent:Okay! Have a great day!
